#### Z2 HIGGS MODEL
$$
\hat{H}_{\text{Z2}} = -g \sum_{i=1}^{L-1} \hat{X}_{i,i+1} + U \sum_{i=1}^{L} \hat{n}_i^2 - J \sum_{i=1}^{L-1} \left( \hat{a}_i^\dagger \hat{Z}_{i,i+1} \hat{a}_{i+1} + \text{h.c.} \right),
$$

For $L=2$ sites this is $\hat{H}_{\text{Z2}} = -g \hat{X}_{1,2} + U\left(\hat{n}_1^2 + \hat{n}_2^2\right) - J\left(\hat{a}_1^\dagger \hat{Z}_{1,2} \hat{a}_2 + \hat{a}_2^\dagger \hat{Z}_{1,2} \hat{a}_1\right)$.

To implement this in an oscillator-qubit architecture, we can do the following:
- First term just corresponds to a qubit rotation
- Hopping term is a conditional beamsplitter between the two sites
- Onsite interaction term can be implemented using a $\text{SNAP}_i(\theta)$ gate (Choosing $\theta_k = U_{ik}^2 t$, where $k$ is the photon number corresponding to a particular Fock state, implements the onsite interaction $\hat{n}_i^2$ on the $i$ th mode.
)

In [ ]:
import sys
import os

# Adjust the path based on your directory structure
module_path = os.path.abspath(os.path.join('..', '..', '..'))  # Moves three directories up
if module_path not in sys.path:
    sys.path.append(module_path)
    
# Now you can import c2qa and other modules from bosonic-qiskit
import c2qa
import qiskit
import numpy as np
import c2qa.util as util

In [ ]:
def hopping_term(circuit, qbr, qb_index, qmr, mode1, mode2, j, delta_t):

    """
    Applies the hopping term as a conditional beam splitter.

    Parameters:
        circuit (QuantumCircuit): The quantum circuit to modify.
        qbr (QuantumRegister): The quantum register containing the gauge field qubits.
        qb_index (int): The index of the qubit in the quantum register acting as the control.
        qmr (QumodeRegister): The quantum register containing the bosonic modes.
        mode1 (int): The index of the first bosonic mode.
        mode2 (int): The index of the second bosonic mode.
        j (float): Coupling constant for the hopping term.
        delta_t (float): Trotter time step.
    """
    theta = -1j * j * delta_t
    circuit.cv_c_bs(theta, qmr[mode1], qmr[mode2], qbr[qb_index])

def e_field(circuit, qbr, qb_index, g, delta_t):
    """
    Applies the electric field term to a specific qubit in the gauge field qubit register.

    Parameters:
        circuit (QuantumCircuit): The quantum circuit to modify.
        qbr (QuantumRegister): The quantum register containing the gauge field qubits.
        qb_index (int): The index of the qubit in the quantum register.
        g (float): Coupling constant for the electric field term.
        delta_t (float): Trotter time step.
    """
    theta = 2 * g * delta_t
    circuit.rx(theta, qbr[qb_index])



